In [ ]:
import pandas as pd

In [ ]:
results_df = pd.read_csv('data/1976-2016-president.csv')

In [ ]:
results_df.head(10)

##Creating a dictionary of state-state abbr entries to be used across the notebook
states = results_df[results_df.year.isin([2016])]
states = states.filter(items = ['state','state_po'])
states = states.drop_duplicates()
states = states.set_index('state')
swing_states = ['CO','FL','MI','IA','MN','NV','NH','NC','OH','PA','VA','WI','AZ','GA']

In [ ]:
##How we are calculating lean
# For each election we look at the margins per state for the previous two elections and aggregate them
# This will be used as the proxy for how a state has leaned in the near past

##2016 lean
years = [2008, 2012]
parties = ['democrat', 'republican']

lean_2016 = results_df[results_df.year.isin(years)]
#MN democrat party is officially called Minnesota Democratic–Farmer–Labor Party 
#https://en.wikipedia.org/wiki/Minnesota_Democratic%E2%80%93Farmer%E2%80%93Labor_Party
lean_2016 = lean_2016.replace({'party' : 'democratic-farmer-labor'},'democrat') 
lean_2016 = lean_2016[lean_2016.party.isin(parties)]


##Some duplicate entries (probably faithless elector votes)
#lean_2016[(results_df['year']==2004) & (lean_2016['state_po'] == 'MD')]
#lean_2016[(results_df['year']==2016) & (lean_2016['state_po'] == 'AZ')]
#lean_2016[(results_df['year']==2016) & (lean_2016['state_po'] == 'MD')]


lean_2016['vote_share'] = round(100 * (lean_2016['candidatevotes']/lean_2016['totalvotes']),2) 
lean_2016 = lean_2016[lean_2016['vote_share'] > 1]  #Few electors voted for Kasich/Sanders
lean_2016 = lean_2016.filter(items = ['year','state_po','party','vote_share'])

lean_2016 = lean_2016.set_index(['year','state_po','party'],verify_integrity=True)

lean_2016 = lean_2016.unstack(level='party')
lean_2016['partisan_lean'] = lean_2016['vote_share']['democrat'] - lean_2016['vote_share']['republican'] 

lean_by_state_2016 = lean_2016.groupby('state_po')['partisan_lean'].mean().to_frame()
lean_by_state_2016.sort_values(by=['partisan_lean'])
lean_by_state_2016
lean_by_state_2016['year']  = 2016

##2012 lean
years = [2004, 2008]
parties = ['democrat', 'republican']

lean_2012 = results_df[results_df.year.isin(years)]
#MN democrat party is officially called Minnesota Democratic–Farmer–Labor Party 
#https://en.wikipedia.org/wiki/Minnesota_Democratic%E2%80%93Farmer%E2%80%93Labor_Party
lean_2012 = lean_2012.replace({'party' : 'democratic-farmer-labor'},'democrat') 
lean_2012 = lean_2012[lean_2012.party.isin(parties)]

lean_2012['vote_share'] = round(100 * (lean_2012['candidatevotes']/lean_2012['totalvotes']),2) 
lean_2012 = lean_2012[lean_2012['vote_share'] > 1]  #Few electors voted for Kasich/Sanders
lean_2012 = lean_2012.filter(items = ['year','state_po','party','vote_share'])

lean_2012 = lean_2012.set_index(['year','state_po','party'],verify_integrity=True)

lean_2012 = lean_2012.unstack(level='party')
lean_2012['partisan_lean'] = lean_2012['vote_share']['democrat'] - lean_2012['vote_share']['republican'] 

lean_by_state_2012 = lean_2012.groupby('state_po')['partisan_lean'].mean().to_frame()
lean_by_state_2012.sort_values(by=['partisan_lean'])
lean_by_state_2012['year']  = 2012

lean_by_state_2012

##2004 lean
years = [2000, 2004]
parties = ['democrat', 'republican']

lean_2008 = results_df[results_df.year.isin(years)]
#MN democrat party is officially called Minnesota Democratic–Farmer–Labor Party 
#https://en.wikipedia.org/wiki/Minnesota_Democratic%E2%80%93Farmer%E2%80%93Labor_Party
lean_2008 = lean_2008.replace({'party' : 'democratic-farmer-labor'},'democrat') 
lean_2008 = lean_2008[lean_2008.party.isin(parties)]

lean_2008['vote_share'] = round(100 * (lean_2008['candidatevotes']/lean_2008['totalvotes']),2) 
lean_2008 = lean_2008[lean_2008['vote_share'] > 1]  #Few electors voted for Kasich/Sanders
lean_2008 = lean_2008.filter(items = ['year','state_po','party','vote_share'])

lean_2008 = lean_2008.set_index(['year','state_po','party'],verify_integrity=True)

lean_2008 = lean_2008.unstack(level='party')
lean_2008['partisan_lean'] = lean_2008['vote_share']['democrat'] - lean_2008['vote_share']['republican'] 

lean_by_state_2008 = lean_2008.groupby('state_po')['partisan_lean'].mean().to_frame()
lean_by_state_2008.sort_values(by=['partisan_lean'])

lean_by_state_2008['year']  = 2008

lean_by_state_2008

lean_by_state = lean_by_state_2016.append(lean_by_state_2012)
lean_by_state = lean_by_state.append(lean_by_state_2008)
lean_by_state = lean_by_state.reset_index()
lean_by_state = lean_by_state.set_index(['state_po','year'])
lean_by_state

In [ ]:
'''
NOT USING EVENT DATA, TINY SAMPLE SIZE AND NOT GREAT RESULTS
dem_event_2016 = pd.read_csv('2016_clinton_kaine_campaign_events.csv')
rep_event_2016 = pd.read_csv('2016_trump_pence_events.csv')

dem_event_2016 = dem_event_2016.set_index('state')
dem_event_2016 = dem_event_2016.rename(columns={"event_count": "dem_event_count"}, errors="raise")
rep_event_2016 = rep_event_2016.set_index('state')
rep_event_2016 = rep_event_2016.rename(columns={"event_count": "rep_event_count"}, errors="raise")

dem_event_2016 = dem_event_2016.join(states)
dem_event_2016 = dem_event_2016.set_index('state_po')

rep_event_2016 = rep_event_2016.join(states)
rep_event_2016 = rep_event_2016.set_index('state_po')

lean_by_state_2016 = lean_by_state_2016.join(dem_event_2016)
lean_by_state_2016 = lean_by_state_2016.join(rep_event_2016)

lean_by_state_2016['dem_event_count'] = lean_by_state_2016['dem_event_count'].fillna(0)
lean_by_state_2016['rep_event_count'] = lean_by_state_2016['rep_event_count'].fillna(0)

lean_by_state_2016

import matplotlib.pyplot as plt

swing_state_lean_2016 = lean_by_state_2016.loc[swing_states]

swing_state_lean_2016['event_count_diff'] = swing_state_lean_2016['dem_event_count'] - swing_state_lean_2016['rep_event_count']

plt.scatter(swing_state_lean_2016['event_count_diff'], swing_state_lean_2016['margin'] - swing_state_lean_2016['partisan_lean'], color='green')
#plt.title('rep_event_count Vs margin_of_victory', fontsize=14)
plt.xlabel('rep_event_count', fontsize=14)
plt.ylabel('margin', fontsize=14)
plt.grid(True)
plt.show()

swing_state_lean_2016

'''

In [ ]:

urban_rural_pop_2010 = pd.read_csv('data/census_urban_rural_population.csv')

urban_rural_pop_2010 = urban_rural_pop_2010.filter(items = ['NAME','H002001','H002002'])

urban_rural_pop_2010 = urban_rural_pop_2010.drop(0,axis=0)


urban_rural_pop_2010 = urban_rural_pop_2010.rename(columns={"NAME" : "state",
                                                  "H002001": "total_population", 
                                                  "H002002": "urban_population" })

urban_rural_pop_2010 = urban_rural_pop_2010[urban_rural_pop_2010.state != 'Puerto Rico'] # :( sorry PR, no statehood for you 

urban_rural_pop_2010 = urban_rural_pop_2010.set_index('state')
urban_rural_pop_2010 = urban_rural_pop_2010.join(states)

urban_rural_pop_2010 = urban_rural_pop_2010.set_index('state_po')

urban_rural_pop_2010 = urban_rural_pop_2010.astype({'total_population': 'int32', 'urban_population' : 'int32'})
urban_rural_pop_2010["pct_urban"] = urban_rural_pop_2010['urban_population'] / urban_rural_pop_2010['total_population']


In [ ]:

##US urbanization growth by year (https://www.statista.com/statistics/269967/urbanization-in-the-united-states/)
#2008 - 80.44 - -0.33
#2010 - 80.77 - 0
#2012 - 81.12 - 0.35
#2016 - 81.86 - 0.89
#2019 - 82.46 - 1.69

#use this data to interpolate urbanization percent growth by year

urban_rural_pop_2008 = urban_rural_pop_2010
urban_rural_pop_2008 = urban_rural_pop_2008.drop(['total_population','urban_population'],axis=1)
urban_rural_pop_2008 = urban_rural_pop_2008["pct_urban"] - 0.0033
urban_rural_pop_2008 = urban_rural_pop_2008.to_frame()
urban_rural_pop_2008['year'] = 2008

urban_rural_pop_2012 = urban_rural_pop_2010
urban_rural_pop_2012 = urban_rural_pop_2012.drop(['total_population','urban_population'],axis=1)
urban_rural_pop_2012 = urban_rural_pop_2012["pct_urban"] + 0.0035
urban_rural_pop_2012 = urban_rural_pop_2012.to_frame()
urban_rural_pop_2012['year'] = 2012

urban_rural_pop_2016 = urban_rural_pop_2010
urban_rural_pop_2016 = urban_rural_pop_2016.drop(['total_population','urban_population'],axis=1)
urban_rural_pop_2016 = urban_rural_pop_2016["pct_urban"] + 0.0089
urban_rural_pop_2016 = urban_rural_pop_2016.to_frame()
urban_rural_pop_2016['year'] = 2016

urban_rural_pop = urban_rural_pop_2008.append(urban_rural_pop_2012)
urban_rural_pop = urban_rural_pop.append(urban_rural_pop_2016)

urban_rural_pop = urban_rural_pop.reset_index()

urban_rural_pop = urban_rural_pop.set_index(['state_po','year'])

urban_rural_pop

In [ ]:
## ACS Education level data
## 2010 (use for 2008 election)

acs_education_2010 = pd.read_csv('data/acs_2010_education_survey.csv')

acs_education_2010 = acs_education_2010.filter(items = ['NAME',
                                                        'S1501_C01_015E']) ##pct bachelors degree or higher 25+                                             
acs_education_2010 = acs_education_2010.drop(0,axis=0)

acs_education_2010 = acs_education_2010.rename(columns={"NAME" : "state",
                                                      "S1501_C01_015E": "pct_college_educated"})
                                               
acs_education_2010 = acs_education_2010[acs_education_2010.state != 'Puerto Rico'] # :(   
acs_education_2010 = acs_education_2010.astype({'pct_college_educated' : 'float32'})
                                               
acs_education_2010['pct_college_educated'] = acs_education_2010['pct_college_educated'] / 100

acs_education_2010 = acs_education_2010.set_index('state')
acs_education_2010 = acs_education_2010.join(states)
acs_education_2010 = acs_education_2010.set_index('state_po')


acs_education_2010['year'] = 2008 
acs_education_2010 = acs_education_2010.reset_index()

acs_education_2010

## 2012 (use for 2012 election)
acs_education_2012 = pd.read_csv('data/acs_2012_education_survey.csv')

acs_education_2012 = acs_education_2012.filter(items = ['NAME',
                                                        'S1501_C01_015E']) ##pct bachelors degree or higher 25+
                                         
acs_education_2012 = acs_education_2012.drop(0,axis=0)

acs_education_2012 = acs_education_2012.rename(columns={"NAME" : "state",
                                                      "S1501_C01_015E": "pct_college_educated"})

acs_education_2012 = acs_education_2012[acs_education_2012.state != 'Puerto Rico'] # :(       
acs_education_2012 = acs_education_2012.astype({'pct_college_educated' : 'float32'})
                                               
acs_education_2012['pct_college_educated'] = acs_education_2012['pct_college_educated'] / 100

acs_education_2012 = acs_education_2012.set_index('state')
acs_education_2012 = acs_education_2012.join(states)
acs_education_2012 = acs_education_2012.set_index('state_po')

acs_education_2012['year'] = 2012 
acs_education_2012 = acs_education_2012.reset_index()


## 2016 (use for 2016 election)
acs_education_2016 = pd.read_csv('data/acs_2016_education_survey.csv')


acs_education_2016 = acs_education_2016.filter(items = ['NAME',
                                                        'S1501_C01_006E', #pop 25+
                                                        'S1501_C01_012E', #bachelors degree 25+
                                                        'S1501_C01_013E']) #grad degree 25+

acs_education_2016 = acs_education_2016.drop(0,axis=0)

acs_education_2016 = acs_education_2016.rename(columns={"NAME" : "state",
                                                  "S1501_C01_006E": "pop_25_plus",
                                                  "S1501_C01_012E": "pop_25_plus_bachelors",
                                                  "S1501_C01_013E": "pop_25_plus_graduate"})


acs_education_2016 = acs_education_2016.astype({'pop_25_plus' : 'int32',
                                                  'pop_25_plus_bachelors' : 'int32',
                                                  'pop_25_plus_graduate' : 'int32'})



acs_education_2016['college_educated_pop'] = acs_education_2016['pop_25_plus_bachelors'] + acs_education_2016['pop_25_plus_graduate']

acs_education_2016['pct_college_educated'] = acs_education_2016['college_educated_pop'] / acs_education_2016['pop_25_plus']

acs_education_2016 = acs_education_2016[acs_education_2016.state != 'Puerto Rico'] # :(

acs_education_2016 = acs_education_2016.set_index('state')
acs_education_2016 = acs_education_2016.join(states) 
acs_education_2016 = acs_education_2016.set_index('state_po')
acs_education_2016 = acs_education_2016.reset_index()

acs_education_2016['year'] = 2016 #using 2015 data for 2016 results

acs_education = acs_education_2010.append(acs_education_2012)
acs_education = acs_education.append(acs_education_2016)
acs_education = acs_education.reset_index()
acs_education = acs_education.drop(['pop_25_plus','pop_25_plus_bachelors','pop_25_plus_graduate','college_educated_pop'], axis=1)

acs_education = acs_education.set_index(['state_po','year'])

acs_education

In [ ]:
## This is our result data i.e. what was the actual margin of victory in that election cycle

result_margins = results_df[results_df.year.isin([2008,2012,2016])]
result_margins = result_margins.replace({'party' : 'democratic-farmer-labor'},'democrat') 

result_margins = result_margins[result_margins.party.isin(parties)]
result_margins['vote_share'] = round(100 * (result_margins['candidatevotes']/result_margins['totalvotes']),2) 
result_margins = result_margins[result_margins['vote_share'] > 1] ##There were a few people who voted for Sanders/Kasich in 2016

result_margins = result_margins.filter(items = ['year','state_po','party','vote_share'])

result_margins = result_margins.set_index(['year','state_po','party'],verify_integrity=True)

result_margins = result_margins.unstack(level='party')

result_margins['margin'] = result_margins['vote_share']['democrat'] - result_margins['vote_share']['republican'] 

result_margins = result_margins['margin']
result_margins.to_frame()
result_margins = result_margins.reset_index()

result_margins = result_margins.set_index(['state_po','year'])
pd.set_option('display.max_rows', 153)
result_margins

In [ ]:
final_data_set = lean_by_state.join(urban_rural_pop)
final_data_set = final_data_set.join(acs_education)
final_data_set = final_data_set.join(result_margins)

final_data_set

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(final_data_set['pct_urban'], final_data_set['partisan_lean'], color='green')
#plt.title('rep_event_count Vs margin_of_victory', fontsize=14)
plt.xlabel('pct_urban', fontsize=14)
plt.ylabel('partisan_lean', fontsize=14)
plt.grid(True)
plt.show()

final_data_set.sort_values(by=['pct_urban'])

In [ ]:
plt.scatter(final_data_set['pct_urban'], final_data_set['partisan_lean'], color='blue')
#plt.title('rep_event_count Vs margin_of_victory', fontsize=14)
plt.xlabel('pct_college_educated', fontsize=14)
plt.ylabel('partisan_lean', fontsize=14)
plt.grid(True)
plt.show()

final_data_set.sort_values(by=['pct_college_educated'])

In [ ]:
from sklearn import linear_model
#import statsmodels.api as sm

#TODO: split into training and test
X = final_data_set[['partisan_lean','pct_urban','pct_college_educated']] 
Y = final_data_set['margin']

regr = linear_model.LinearRegression()
regr.fit(X, Y)


In [ ]:
years = [2008, 2012]
parties = ['democrat', 'republican']

lean_2020 = results_df[results_df.year.isin(years)]
#MN democrat party is officially called Minnesota Democratic–Farmer–Labor Party 
#https://en.wikipedia.org/wiki/Minnesota_Democratic%E2%80%93Farmer%E2%80%93Labor_Party
lean_2020 = lean_2020.replace({'party' : 'democratic-farmer-labor'},'democrat') 
lean_2020 = lean_2020[lean_2020.party.isin(parties)]

lean_2020['vote_share'] = round(100 * (lean_2020['candidatevotes']/lean_2020['totalvotes']),2) 
lean_2020 = lean_2020[lean_2020['vote_share'] > 1]  #Few electors voted for Kasich/Sanders
lean_2020 = lean_2020.filter(items = ['year','state_po','party','vote_share'])

lean_2020 = lean_2020.set_index(['year','state_po','party'],verify_integrity=True)

lean_2020 = lean_2020.unstack(level='party')
lean_2020['partisan_lean'] = lean_2020['vote_share']['democrat'] - lean_2020['vote_share']['republican'] 

lean_2020 = lean_2020.groupby('state_po')['partisan_lean'].mean().to_frame()
lean_2020.sort_values(by=['partisan_lean'])
lean_2020['year']  = 2020

lean_2020

urban_rural_pop_2020 = urban_rural_pop_2010
urban_rural_pop_2020 = urban_rural_pop_2020.drop(['total_population','urban_population'],axis=1)
urban_rural_pop_2020 = urban_rural_pop_2020["pct_urban"] + 0.0089
urban_rural_pop_2020 = urban_rural_pop_2020.to_frame()
urban_rural_pop_2020['year'] = 2020

urban_rural_pop_2020 = urban_rural_pop_2020.reset_index()
urban_rural_pop_2020 = urban_rural_pop_2020.set_index(['state_po','year'])

urban_rural_pop_2020

## 2019 (use for 2020 election)
acs_education_2020 = pd.read_csv('data/acs_2019_education_survey.csv')


acs_education_2020 = acs_education_2020.filter(items = ['NAME',
                                                        'S1501_C01_006E', #pop 25+
                                                        'S1501_C01_012E', #bachelors degree 25+
                                                        'S1501_C01_013E']) #grad degree 25+

acs_education_2020 = acs_education_2020.drop(0,axis=0)

acs_education_2020 = acs_education_2020.rename(columns={"NAME" : "state",
                                                  "S1501_C01_006E": "pop_25_plus",
                                                  "S1501_C01_012E": "pop_25_plus_bachelors",
                                                  "S1501_C01_013E": "pop_25_plus_graduate"})


acs_education_2020 = acs_education_2020.astype({'pop_25_plus' : 'int32',
                                                  'pop_25_plus_bachelors' : 'int32',
                                                  'pop_25_plus_graduate' : 'int32'})



acs_education_2020['college_educated_pop'] = acs_education_2020['pop_25_plus_bachelors'] + acs_education_2020['pop_25_plus_graduate']

acs_education_2020['pct_college_educated'] = acs_education_2020['college_educated_pop'] / acs_education_2020['pop_25_plus']

acs_education_2020 = acs_education_2020[acs_education_2020.state != 'Puerto Rico'] # :(

acs_education_2020 = acs_education_2020.set_index('state')
acs_education_2020 = acs_education_2020.join(states) 
acs_education_2020 = acs_education_2020.set_index('state_po')
acs_education_2020 = acs_education_2020.reset_index()

acs_education_2020['year'] = 2020 #using 2015 data for 2016 results
acs_education_2020 = acs_education_2020.drop(['pop_25_plus','pop_25_plus_bachelors','pop_25_plus_graduate','college_educated_pop'], axis=1)
acs_education_2020 = acs_education_2020.set_index(['state_po','year'])
acs_education_2020


urban_rural_pop_2020

In [ ]:
import numpy as np
predict_data_set = lean_2020.join(urban_rural_pop_2020)
predict_data_set = predict_data_set.join(acs_education_2020)

for index, row in predict_data_set.iterrows():
    sample = np.array([row['partisan_lean'],row['pct_urban'],row['pct_college_educated']]).reshape(1,-1)
    margin = regr.predict(sample)
    print("Democrat margin for state " + str(index[0]) + " expected to be " + str(margin[0]))
